<a href="https://colab.research.google.com/github/Tati-AG/TP1_Mineracao_de_Dados/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive

drive.mount ('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np

In [ ]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.basics.tsv'

data_types = defaultdict (lambda: str, {5:'Int64', 6:'Int64'})

title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
                              converters = {7: lambda x: int(x) if x.isdigit() else pd.NA})

In [ ]:
movie_basics = title_basics.loc[np.where ((title_basics['titleType'].values == 'movie'))]

In [ ]:
genres_by_movie = movie_basics['genres'].str.split(',').explode()

In [ ]:
movie_genres = pd.DataFrame ({'genre' : genres_by_movie.unique()})

In [ ]:
movie_genre_bridge = pd.DataFrame ({'tconst' : movie_basics.loc[genres_by_movie.index]['tconst'].values,
                                    'genre' : genres_by_movie.values})

In [ ]:
movie_basics.drop('genres', axis='columns', inplace=True)
del title_basics
del genres_by_movie

In [ ]:
movie_basics.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_basics.csv', index = False)
movie_genres.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_genres.csv', index = False)
movie_genre_bridge.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_genre_bridge.csv', index = False)

In [ ]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.ratings.tsv'

data_types = {0:str, 1:'Float64', 2:'Int64'}

title_ratings = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [ ]:
movie_ratings = title_ratings.loc[np.where ((title_ratings['tconst'].isin(movie_basics['tconst'])))]

In [ ]:
movie_ratings.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_ratings.csv', index = False)

In [ ]:
del title_ratings

In [ ]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.principals.tsv'

data_types = defaultdict (lambda: str, {1:'Int64'})

title_principals = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [ ]:
movie_principals = title_principals.loc[np.where ((title_principals['tconst'].isin(movie_basics['tconst'])))]

In [ ]:
movie_principals.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_principals.csv', index = False)

In [ ]:
del title_principals

In [ ]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.crew.tsv'

data_types = defaultdict (lambda: str, {1:'Int64'})

title_crew = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [ ]:
movie_crew = title_crew.loc[np.where ((title_crew['tconst'].isin(movie_basics['tconst'])))]

In [ ]:
director_by_movie = movie_crew['directors'].str.split(',').explode()
movie_directors = pd.DataFrame ({'tconst' : movie_crew.loc[director_by_movie.index]['tconst'],
                                 'role' : 'director',
                                 'nconst' : director_by_movie.values})

In [ ]:
writer_by_movie = movie_crew['writers'].str.split(',').explode()
movie_writers = pd.DataFrame ({'tconst' : movie_crew.loc[writer_by_movie.index]['tconst'],
                               'role' : 'writer',
                               'nconst' : writer_by_movie.values})

In [ ]:
movie_crew.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_crew.csv', index = False)

In [ ]:
tsv_file = '/content/drive/MyDrive/imdb_data/name.basics.tsv'

data_types = defaultdict (lambda: str, {1:'Int64'})

name_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [ ]:
movie_crew_basics = name_basics.loc[np.where ((name_basics['nconst'].isin(title_crew['nconst'])))]
movie_principals_basics = name_basics.loc[np.where ((name_basics['nconst'].isin(title_principals['nconst'])))]

In [ ]:
movie_crew_basics.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_crew_basics.csv', index = False)
movie_principals_basics.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_principals_basics.csv', index = False)